In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial

import numpy as np
import pandas as pd

!pip install tqdm
from tqdm import tqdm

import sys
import os
import re
import sys
import logging
import numpy as np
from scipy import spatial
import pandas as pd

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#%%
# LASER
!pip install laserembeddings
!python -m laserembeddings download-models
from laserembeddings import Laser

     |████████████████████████████████| 860kB 4.3MB/s 
     |████████████████████████████████| 51kB 4.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp37-none-any.whl size=883999 sha256=f7ff5435326483e128edcc89e5e4ba410abec8caa3e3005922146071c3ab6cb4
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses

✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [7]:
# BERT
!pip install bert_score
from bert_score import score

     |████████████████████████████████| 61kB 2.6MB/s 
     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 3.3MB 18.9MB/s 


In [8]:
# USE
!pip install tensorflow_text 
import tensorflow_text
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/2"
use_model = hub.load(module_url)

     |████████████████████████████████| 3.4MB 4.2MB/s 


2021-04-24 17:35:26,620 : INFO : Using /tmp/tfhub_modules to cache modules.
2021-04-24 17:35:26,625 : INFO : Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/2'.
2021-04-24 17:35:30,222 : INFO : Downloaded https://tfhub.dev/google/universal-sentence-encoder-multilingual/2, Total size: 266.75MB
2021-04-24 17:35:30,224 : INFO : Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/2'.


In [9]:
# FastText
!pip install ufal.udpipe
!pip install wget

import wget
import gensim
from ufal.udpipe import Model, Pipeline

modelfile = wget.download('https://rusvectores.org/static/models/udpipe_syntagrus.model')
udpipe_model = Model.load(modelfile)
process_pipeline = Pipeline(udpipe_model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

from gensim.models import FastText
# fasttext_model = FastText.load_fasttext_format("./drive/MyDrive/NLP/fasttext/wiki.ru")
# fasttext_model.wv.save('./drive/MyDrive/NLP/fasttext/fasttext_gensim_1.model')
fasttext_model = gensim.models.KeyedVectors.load("./drive/MyDrive/NLP/fasttext/fasttext_gensim_1.model")




     |████████████████████████████████| 307kB 4.3MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626632 sha256=27a15da155c6969ab3a23390580271235d6e0172e0cf3e63da0772282bc4ab97
  Stored in directory: /root/.cache/pip/wheels/0c/9d/db/6d3404c33da5b7adb6c6972853efb6a27649d3ba15f7e9bebb
Successfully built ufal.udpipe
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=5bed54e07215ba2d5f442cf254ddee00b146e76bbdb0fa3decd3b0f7a6c23c94
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


2021-04-24 17:37:37,066 : INFO : 'pattern' package not found; tag filters are not available for English
2021-04-24 17:37:43,590 : INFO : loading Word2VecKeyedVectors object from ./drive/MyDrive/NLP/fasttext/fasttext_gensim_1.model
2021-04-24 17:37:50,390 : INFO : loading vectors from ./drive/MyDrive/NLP/fasttext/fasttext_gensim_1.model.vectors.npy with mmap=None
2021-04-24 17:38:09,992 : INFO : loading vectors_ngrams from ./drive/MyDrive/NLP/fasttext/fasttext_gensim_1.model.vectors_ngrams.npy with mmap=None
2021-04-24 17:38:47,028 : INFO : setting ignored attribute vectors_norm to None
2021-04-24 17:38:47,037 : INFO : setting ignored attribute vectors_vocab_norm to None
2021-04-24 17:38:47,048 : INFO : setting ignored attribute vectors_ngrams_norm to None
2021-04-24 17:38:47,051 : INFO : setting ignored attribute buckets_word to None
2021-04-24 17:38:47,055 : INFO : loaded ./drive/MyDrive/NLP/fasttext/fasttext_gensim_1.model


In [10]:
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.
stop_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [109]:
#%%

def get_avg_vector(processed_text, model, num_features=300):
  feature_vec = np.zeros((num_features, ), dtype='float32')
  n_words = 0
  for i in processed_text:
    if i in model:
      n_words += 1
      feature_vec = np.add(feature_vec, model[i])
  if (n_words > 0):
    feature_vec = np.divide(feature_vec, n_words)
  return feature_vec

def cosine_similarity(v1, v2):
  return 1 - spatial.distance.cosine(v1, v2)

def get_similarity(text1, text2, model):
  v1 = get_avg_vector(text1, model)
  v2 = get_avg_vector(text2, model)

  return cosine_similarity(v1, v2)

def get_missing_percent(processed_text, model):
  vocab_missings = 0
  missing_words = []
  max_count = len(processed_text)
  for i in processed_text[:max_count]:
    if i in model: 
      vocab_missings += 1
      missing_words.append(i)
  result = (vocab_missings/max_count)*100
  return result, missing_words

def num_replace(word):
    newtoken = "x" * len(word)
    return newtoken
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False  # переменная для запоминания того, что нам встретилось имя собственное
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
      if len(t) != 10: # если список короткий — строчка не содержит разбора, пропускаем
          continue
      (word_id,token,lemma,pos,xpos,feats,head,deprel,deps,misc) = t
      if not lemma or not token: # если слово пустое — пропускаем
          continue
      if pos in entities: # здесь отдельно обрабатываем имена собственные — они требуют особого обращения
          if '|' not in feats:
              tagged_propn.append('%s_%s' % (lemma, pos))
              continue
          morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
          if 'Case' not in morph or 'Number' not in morph:
              tagged_propn.append('%s_%s' % (lemma, pos))
              continue
          if not named:
              named = True
              mem_case = morph['Case']
              mem_number = morph['Number']
          if morph['Case'] == mem_case and morph['Number'] == mem_number:
              memory.append(lemma)
              if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                  named = False
                  past_lemma = '::'.join(memory)
                  memory = []
                  tagged_propn.append(past_lemma + '_PROPN ')
          else:
              named = False
              past_lemma = '::'.join(memory)
              memory = []
              tagged_propn.append(past_lemma + '_PROPN ')
              tagged_propn.append('%s_%s' % (lemma, pos))
      else:
          if not named:
              if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                  lemma = num_replace(token)
              tagged_propn.append('%s_%s' % (lemma, pos))
          else:
              named = False
              past_lemma = '::'.join(memory)
              memory = []
              tagged_propn.append(past_lemma + '_PROPN ')
              tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct: # обрабатываем случай, когда пользователь попросил не сохранять пунктуацию (по умолчанию она сохраняется)
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

def preprocess(sentence):
    return ' '.join([w for w in sentence.lower().split() if w not in stop_words])

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model', keep_pos=True):
   lines = text.split('\n')
   tagged = []
   for line in lines[:2]:
      line = preprocess(line)
      # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
      output = process(process_pipeline, text=line, keep_pos=keep_pos)
      # tagged_line = ' '.join(output)
      tagged.append(output)
   return [item for sublist in tagged for item in sublist]

def calculate_cosine_similarity(x, y):
  assert x.shape[0] == y.shape[0]
  simularities = []
  for i in range(x.shape[0]):
    try:
      xi = x[i].toarray()
      yi = y[i].toarray()
    except Exception:
      xi = np.array(x[i])
      yi = np.array(y[i])
    if xi.shape[0] != 1:
      xi = xi.reshape(1, -1)
      yi = yi.reshape(1, -1)

    sim = cosine_similarity(xi,yi)
    if math.isnan(sim):
      sim = 0.
    simularities.append(sim)
  return simularities

#%%
#%%
#%%
#%%
#%%

def calculate_bert_distance(data, field="bert_distance"):
    print("==== Bert calculation start ====")
    origins = data["original"].tolist()
    targets = data["scored_text"].tolist()
    P, R, F1 = score(origins, targets, lang="ru", verbose=True)
    data[field] = R
    print("==== Bert calculation end ====")

def calculate_use_distance(data, field="use_distance"):
    print("==== USE calculation start ====")
    origins = data["original"].tolist()
    targets = data["scored_text"].tolist()
    original_embeddings = use_model(origins)['outputs']
    targets_embeddings = use_model(targets)['outputs']
    corr = calculate_cosine_similarity(original_embeddings, targets_embeddings)
    data[field] = corr
    print("==== USE calculation end ====")

def calculate_tfidf_distance(data, field="tfidf_distance"):
  print("==== TfIdf calculation start ====")
  original = data["original"].tolist()
  targets = data["scored_text"].tolist()

  vectorizer = TfidfVectorizer()
  original_vectors = vectorizer.fit_transform(original)
  target_vectors = vectorizer.transform(targets)
  corr = calculate_cosine_similarity(original_vectors, target_vectors)
  data[field] = corr
  print("==== TfIdf calculation start ====")

def calculate_laser_distance(data, field="laser_distance"):
  print("==== Laser calculation start ====")
  original = data["original"].tolist()
  targets = data["scored_text"].tolist()

  laser = Laser()
  original_vectors = laser.embed_sentences(original, lang='ru')
  target_vectors = laser.embed_sentences(targets, lang='ru')
  corr = calculate_cosine_similarity(original_vectors, target_vectors)
  data[field] = corr
  print("==== Laser calculation end ====")

def calculate_fasttext_sim(data, field = "fasttext_distance"):
  print("==== FastText calculation start ====")
  model = fasttext_model
  original = data["original"].tolist()
  targets = data["scored_text"].tolist()
  sims = []

  missed_words = set()
  missing_percent = 0

  for i in tqdm(range(len(original))):
    origin = original[i]
    target = targets[i]
    origin = tag_ud(text=origin, modelfile=modelfile, keep_pos=False)
    target = tag_ud(text=target, modelfile=modelfile, keep_pos=False)
    sim = get_similarity(origin, target, model)
    sims.append(sim)

    percent, missing = get_missing_percent(origin, model)
    missing_percent += percent
    missed_words.update(missing)

  missing_percent /= len(original)
  data[field] = sims
  print("==== FastText calculation end ====")
  return sims, missing_percent, missed_words

#%%


def calculate_distance(input="./drive/MyDrive/NLP/input/data_with_semi_preproc_without_stopwords.csv",
                       output="./drive/MyDrive/NLP/output/semi_preproc_without_stopwords_data_with_distance.csv"):
    data = pd.read_csv(input)
    data = data.dropna()

    calculate_tfidf_distance(data)
    calculate_laser_distance(data)
    calculate_use_distance(data)
    calculate_fasttext_sim(data)
    calculate_bert_distance(data)
    data.to_csv(output, index=False)
    return data


In [110]:
calculate_distance()

==== TfIdf calculation start ====
==== TfIdf calculation start ====
==== Laser calculation start ====


/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


==== Laser calculation end ====
==== USE calculation start ====


  1%|          | 1/161 [00:00<00:21,  7.52it/s]

==== USE calculation end ====
==== FastText calculation start ====


100%|██████████| 161/161 [00:09<00:00, 16.45it/s]


==== FastText calculation end ====
==== Bert calculation start ====
calculating scores...
computing bert embedding.



computing greedy matching.



done in 39.39 seconds, 4.09 sentences/sec
==== Bert calculation end ====


,Unnamed: 0,original,scored_text,distance,score,tfidf_distance,laser_distance,use_distance,fasttext_distance,bert_distance
0,0,великий княжество владимирский 1157года — ...,середина xiii век сюзеренитет великий князь вл...,0.923924,5,0.970237,0.937215,0.995279,0.994356,0.898097
1,1,новгородский республика — севернорусский с...,находиться 1245 год сюзеренитет великий князь ...,0.908120,5,0.986404,0.940422,0.990447,0.996687,0.887942
2,2,великий княжество литовский — восточноевроп...,1385 год находиться личный уния королевство по...,0.861883,5,0.985886,0.931532,0.995013,0.997149,0.896861
3,3,великий княжество московский — средневековы...,возвышение москва укрепление авторитет русь сп...,0.901416,5,0.986935,0.913012,0.989068,0.985685,0.905214
4,4,олег — князь новгородский 879 год великий ...,получать власть новгородский земля смерть рюри...,0.916509,5,0.923599,0.923447,0.985559,0.971720,0.889845
...,...,...,...,...,...,...,...,...,...,...
157,157,александр федорович керенский 1917 год свергат...,отцовский сторона предок александр керенский п...,0.901247,3,0.662703,0.778323,0.493093,0.646251,0.863597
158,158,лев борисович каменев 1922 год — заместител...,член цик председатель вцик ссср,0.878858,3,0.591546,0.691548,0.546366,0.434815,0.865166
159,159,яков михайлович свердлов партийный псевдоним а...,прима,0.736985,3,0.235471,0.340857,0.037572,0.243300,0.789153
160,160,состояниеотпатрулированаиосиф виссарионович ст...,маршал советский союз генералиссимус советский...,0.901779,3,0.697200,0.747728,0.633659,0.644360,0.879614


In [91]:
import math
data = pd.read_csv('./drive/MyDrive/NLP/input/data_origin.csv')

original = data["original"].tolist()
targets = data["scored_text"].tolist()

original_vectors = use_model(original)['outputs']
target_vectors = use_model(targets)['outputs']

x = original_vectors
y = target_vectors
assert x.shape[0] == y.shape[0]
simularities = []
for i in range(x.shape[0]):
  xi = np.array(x[i])
  yi = np.array(y[i])
  if xi.shape[0] != 1:
    xi = xi.reshape(1, -1)
    yi = yi.reshape(1, -1)

  sim = cosine_similarity(xi,yi)
  if math.isnan(sim):
    sim = 0.

  simularities.append(sim)
corr = simularities


